In [ ]:
import gstlearn as gl
import gstlearn.test as gt
import gstlearn.document as gdoc
import numpy as np
from IPython.display import Markdown
from scipy.spatial import distance_matrix

In [ ]:
Markdown(gdoc.loadDoc("Kriging.md"))

In [ ]:
Markdown(gdoc.loadDoc("Kriging_Bayes.md"))

In [ ]:
def cova(x,sills=1):
    return np.kron(sills,np.exp(-x/2))

np.random.seed(1234)
A = np.random.normal(size=(3,3))
sills = gl.VectorDouble((A@A.T).reshape(1,-1)[0])
model = gl.Model.createFromParam(gl.ECov.EXPONENTIAL,range = 2.,flagRange=False,sills=sills)

nx = [10,10]

def matriceReduce(m,ind):
    M = gl.MatrixSquareSymmetric(len(ind))
    for i,j in enumerate(ind):
        for k in range(j,len(ind)):
            M.setValue(i,k,m.getValue(j,ind[k]))
    return M

def modelReduce(model,var):
    ctxt=model.getContext()
    ctxt.setNVar(len(var))
    modeln = gl.Model.create(ctxt)
    covlist = model.getCovAnisoList()
    
    for i in range(covlist.getCovaNumber()):
        cova = covlist.getCova(i)
        sills = matriceReduce(cova.getSill(),var)
        covar = gl.CovAniso.createAnisotropicMulti(ctxt,
                                             cova.getType(),
                                             cova.getRanges(),
                                             sills,
                                             cova.getParam(),
                                             cova.getAnisoAngles())
        modeln.addCov(covar)
    return modeln


def createDbIn(ndat,nvar,percent,ndim=2,selDbin=False,measurement_error=False,ndrift = 0,
               flag_isotopic=False,seed=1234):
    db = gl.Db.create()
    np.random.seed(seed)
    for i in range(ndim):
        db["x" +str(i)] = np.random.uniform(size = ndat)
     
    db.setLocators(["x*"],gl.ELoc.X)
        
    indIn = np.arange(ndat)
    if selDbin:
        np.random.shuffle(indIn)
        indIn = np.sort(indIn)
        indIn = indIn[range(int(ndat/2))]
        sel = np.zeros(shape=ndat)
        sel[indIn] = 1
        db["sel"] = sel
        db.setLocator("sel",gl.ELoc.SEL)
      
    #Creation of an heterotopic data set
    indList = [] 
    for i in range(nvar):
        u = np.array([None for i in range(ndat)])
        ind = np.arange(ndat)
        if not flag_isotopic and nvar>1: 
            np.random.shuffle(ind)
            ind = ind[range(int(ndat*percent[i]))]
        ind = np.sort(list(set(ind) & set(indIn)))
        indList += [ind]
        vect = np.array([None for i in range(ndat)])
        vect[ind] = np.random.normal(size = len(ind))
        db["z"+str(i)]=vect
          
    db.setLocators(["z*"],gl.ELoc.Z)
    
    indF = []
    
    for i in range(nvar):
        indF += list(np.array(indList[i]) + ndat * i)
    
    if measurement_error :
        for i in range(nvar):
            db["err"+str(i)] = 0.1 * np.random.uniform(size = ndat)
            
        db.setLocators(["err*"],gl.ELoc.V)
    
    if ndrift>0:
        for i in range(ndrift):
            db["ff" + str(i)] = np.random.normal(size = ndat)
        db.setLocator("ff*",gl.ELoc.F)
    
    return db,indF


def test_covmat(ndat,nx,nvar,percent,model,cova,
                 irf=None,drift=False,measurement_error=False,compute_vars = True,
                 selDbin = True, selDbout = True,flag_isotopic=True,
                 seed=1234,tol=1e-12,eps=1e-3,test = True,verbose=False):
    
    np.random.seed(seed)
    ndrift = 1 if drift else 0
    modeln = modelReduce(model,range(nvar))
    #### Create the description of the case #####
    casetxt = "case:\n"
    
    inter = ""
    if nvar > 1:
        inter = "co-"
        
    if irf is None and drift:
        return 0
    
    if irf is None and not drift:
        casetxt += "- simple "+ inter+ "kriging\n"
    else :
        if irf is not None :
            casetxt += "- KU with drift of degree " + str(irf) + "\n"
        if drift :
            casetxt +="- with external drift\n"
    if nvar > 1:
        casetxt +="- number of covariables for co-kriging " + str(nvar) + "\n"
        if flag_isotopic:
            casetxt += "- isotopic case\n"
        else:
            casetxt += "- heterotopic case\n"
    if measurement_error:
        casetxt += "- with measurement error\n"
    else:
        casetxt += "- without measurement error\n"
    if compute_vars:
        casetxt += "- no dual form\n"
    else:
        casetxt += "- dual\n"
    casetxt += "- selection on Dbin " + str(selDbin) + "\n"
    casetxt += "- selection on Dbout "+ str(selDbout) + "\n"
    casetxt += "- number of data " + str(ndat) + "\n"
    casetxt += "- nx = ["+str(nx[0]) +"," + str(nx[1]) + "]\n"
    
    if verbose:
        print(casetxt)
        
    ##################################################
    db,indF = createDbIn(ndat,nvar,percent,2,selDbin,measurement_error,ndrift,flag_isotopic,seed)
    
    target = gl.DbGrid.create(nx = nx)
   
    indOut = np.arange(target.getSampleNumber())
    
    if selDbout:
        np.random.shuffle(indOut)
        indOut = indOut[range(int(target.getSampleNumber()/2))]
        indOut = np.sort(indOut)
        sel = np.zeros(shape = target.getSampleNumber())
        sel[indOut] = 1
        target["sel"] = sel
        target.setLocator("sel",gl.ELoc.SEL)
                  
    if irf is not None:
        modeln.setDriftIRF(irf)
    
    if drift :
        target["ff"] = np.random.normal(size = target.getSampleNumber())
        
        target.setLocator("ff",gl.ELoc.F)
        modeln.addDrift(gl.DriftF(0))
      
    v = np.array([db["x0"],db["x1"]]).T
    v0 = np.array([target["x1"][indOut],target["x2"][indOut]]).T
    cov = cova(distance_matrix(v,v),modeln.getSillValues(0).toTL())[indF,:][:,indF]
    c0  = cova(distance_matrix(v,v0),modeln.getSillValues(0).toTL())[indF,:]
    #Creation of a db2 without selection to build the complete covariance matrix
    db2 = db.clone()
    db2.setLocator("sel")
    vect = gl.VectorDouble(nvar**2 * db2.getSampleNumber()**2)
    
    target2 = target.clone()
    target2.setLocator("sel")
    covt = modeln.evalCovMatrixSymmetric(db).toTL()
    c0gl = modeln.evalCovMatrix(db,target).toTL()
    
    if measurement_error:
        err = db["err*"].T.reshape(-1,)
        np.fill_diagonal(cov,np.diag(cov)+err[indF])
    
    vect = gl.VectorDouble(nvar**2 * db2.getSampleNumber() * len(indOut))
    
    neigh = gl.NeighUnique()
    
    errcode = 0
    diff = np.max(np.abs(cov-covt))
    if diff > eps:
        errcode = errcode + 1
        print(f"Difference in Cov  = {round(diff,4)}")
    diff = np.max(np.abs(c0-c0gl))
    if diff > eps:
        errcode = errcode + 1
        print(f"Difference in Cov0 = {round(diff,4)}")
    if irf is not None or drift:
        driftd = np.kron(np.eye(nvar),modeln.getDrifts(db2, False))[:,indF]
        driftt = np.kron(np.eye(nvar),modeln.getDrifts(target, True))
        driftdgl = modeln.evalDriftMatrix(db).toTL().T
        drifttgl = modeln.evalDriftMatrix(target).toTL().T
        diff = np.max(np.abs(driftdgl-driftd))
        if diff > eps:
            errcode = errcode + 1
            print(f"Difference in X  = {round(diff,4)}")
        diff = np.max(np.abs(drifttgl-driftt))
        if diff > eps:
            errcode = errcode + 1
            print(f"Difference in X0 = {round(diff,4)}")

    return errcode


In [ ]:
percent = [0.5,0.9,1.]
nerr = 0
ntest = 0
for irf in [None,0,1]:
  for drift in [False,True]:
    for measurement_error in [True, False]:
      for selDbin in [True, False]:
        for selDbout in [True, False]:
          for nx in [[5,5]]:
            for nvar in [1,2,3]:
              isolist = [True]
              if nvar >1 :
                isolist = [True,False]
              for iso in isolist:
                for cv in [False,True]:
                  errcode = test_covmat(40,nx,nvar,percent,
                                        model,cova,compute_vars=cv,
                                        irf=irf,drift=drift,
                                        measurement_error=measurement_error,
                                        selDbin=selDbin,selDbout=selDbout,
                                        flag_isotopic = False,
                                        seed=1234,tol=1e-8,eps=1e-3,verbose=False)
                  nerr = nerr + errcode
                  ntest = ntest + 1
                  
print(ntest,"test(s) have been performed with", nerr, "error(s)")